In [1]:
# Import packages
import os
import random
from PIL import Image
from torch import nn
import torch 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import json 
from src.config import PROCESSED_DATA_DIR, RAW_DATA_DIR
import torch.optim as optim

from model_architectures.v01 import HHP01
from model_architectures.v02 import HHP02

from utils import train_validate_model

image_path = PROCESSED_DATA_DIR / "pizza_hamburger_hotdog_20_percent"
train_dir = image_path / 'train' 
test_dir = image_path / 'test'
valid_dir = image_path / 'valid'

device ='cuda' if torch.cuda.is_available() else 'cpu'

2024-06-21 21:15:37.188 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Git\hamburger-hotdog-pizza-classifier


In [2]:
config_path = 'params.json'

with open(config_path, 'r') as config_file:
   config = json.load(config_file)
#print(json.dumps(config, indent=2))

In [3]:
# Load hyperparams from params.json

learning_rate = config['model_params']['learning_rate']
batch_size = config['model_params']['batch_size']
num_epochs = config['model_params']['num_epochs']
dropout_rate = config['model_params']['dropout_rate']
optimizer = config['model_params']['optimizer']
loss_function = config['model_params']['loss_function']
metrics = config['model_params']['metrics']
output_shape = config['model_params']['output_shape']

In [4]:
# Prepare data into dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])

train_data = ImageFolder(train_dir, transform=transform)
valid_data = ImageFolder(valid_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [5]:
#import torchinfo
#torchinfo.summary(model, input_size=(1, 3, 224, 224))

In [6]:
model = HHP02(input_shape=3, hidden_units=10, output_shape=output_shape).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [7]:
# train_validate_model(num_epochs=100, train_loader=train_loader, valid_loader=valid_loader, model=model, criterion=criterion, optimizer=optimizer, device=device)

In [8]:
image_batch, label_batch = next(iter(train_loader))
image_batch.shape, label_batch.shape


(torch.Size([64, 3, 224, 224]), torch.Size([64]))

In [9]:
from model_architectures.v02 import HHP02
model = HHP02(input_shape=3, hidden_units=10, output_shape=output_shape).to(device)
model(image_batch.to(device))

torch.Size([64, 10, 110, 110])
torch.Size([64, 10, 53, 53])
torch.Size([64, 10, 23, 23])
torch.Size([64, 10, 8, 8])
torch.Size([64, 10, 1, 1])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x10 and 28090x4096)